In [103]:
#
# Read in api key for (newsapi-python) newsapi.org and set global variable
# #--> pip install newsapi-python
#

import requests
import json
from contextlib import closing

# get API key for newsapi.org
APIKEY_FILE = '../newsapi_key.txt'

def read_api_key(api_key_fname):
    """
        read in api key from file. relative path, file just contains api key. return key as string
    """
    with open(api_key_fname) as f:
        api_key = f.read()
    
    return api_key

api_key = read_api_key(APIKEY_FILE)

In [104]:
# Initilize NewsApiClient using api key
# using python package newsapi-python
#--> pip install newsapi-python
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key=api_key)

In [105]:
# Available sources
# get sources from NewsApiClient/newsapi.org
sources_response = newsapi.get_sources()
assert sources_response['status'] == 'ok'

# create a name:id python dictonary / map. The ids can be used for requests to NewsApiClient
name_id_dict = {s['name']:s['id'] for s in sources_response['sources']}
# creat a id:name dictonary
id_name_dict = {v: k for k, v in name_id_dict.items()}

# create a pandas dataframe from this dictonary with id as first column
import pandas as pd

id_name_df = pd.DataFrame.from_dict(data=id_name_dict, orient='index', columns=['name'])
id_name_df.index.name = 'id'
id_name_df

,name
id,
abc-news,ABC News
abc-news-au,ABC News (AU)
aftenposten,Aftenposten
al-jazeera-english,Al Jazeera English
ansa,ANSA.it
argaam,Argaam
ars-technica,Ars Technica
ary-news,Ary News
associated-press,Associated Press


In [106]:
"""
Article Scraper from a given url

requirements: 
	-Newspaper3k, https://github.com/codelucas/newspaper, http://newspaper.readthedocs.io/en/latest/, https://newsapi.org/docs/client-libraries/python
    --> pip install newspaper3k
Notes: 	
	newspaper offers nlp summary
	article.nlp()
	print(article.summary)

"""
# external
from newspaper import Article

def get_full_article(url):
    # does not work for video news sources etc
    article = Article(url)
    article.download()
    article.parse()
    text = article.text
    if len(text) < 10:
        print("warning with article url when extracting full text. Function: get_full_article")
        return None
    return text

# testing
test_article_url = "http://www.foxnews.com/world/2018/08/28/likely-lula-replacement-denies-corruption-charges-in-brazil.html"
print(get_full_article(test_article_url))

next

prev

The man in line to replace jailed presidential candidate Luiz Inacio Lula da Silva as the Brazilian Workers' Party standard-bearer on Tuesday denied accusations of corruption.

Prosecutors accuse Fernando Haddad of receiving indirect payments to his 2012 campaign for Sao Paulo mayor.

Construction company UTC Engenharia got preferential treatment on bids after covering about US$1.6 million of debt associated with Haddad's campaign, according to a former accusation filed Monday. Although Haddad did not request payment directly, he had full control over the scheme, according to prosecutors.

On Tuesday, Haddad responded while campaigning in Rio de Janeiro. He said that he cancelled a multimillion dollar project with a company belonging to the UTC group after an employee alerted him that the company was overcharging the government.

"How is it that a mayor who cancels a corrupt construction project gets put through this instead of being thanked for saving the city tens of mill

In [124]:
# get list of article objects from response from newsapi.org
#--> pip install newsapi-python
#from newsapi import NewsApiClient
#-Newspaper3k, https://github.com/codelucas/newspaper, http://newspaper.readthedocs.io/en/latest/, https://newsapi.org/docs/client-libraries/python
#--> pip install newspaper3k

def get_list_of_article_objects(newsapi_article_response, include_full_articles=True):
    """ given a article response from NewsApiClient return a list of article objects including the source id, title description url and if requested the full text of the article"""
    
    # get array of article json objects/list
    articles = newsapi_article_response['articles']
    
    # for article in articles look at 'source', 'title', 'description', 'url'
    # extract article sources titles descriptions and links to urls of actual articles
    article_objects = []
    for a in articles:
        
        article_object = {
            'news_source_id' : a['source']['id'], # article news source id
            'title' : a['title'], # article title 
            'description' : a['description'], # breif article description
            'url' : a['url'], # full article url
        }
        
        if include_full_articles:
            article_object['text'] = get_full_article(a['url']) # get full article text using Newspaper3k -- often None
        
        if None not in article_object.values():
            article_objects.append(article_object)
        
    return article_objects

In [125]:
# or by using python package
# -- https://github.com/mattlisiv/newsapi-python
#--> pip install newsapi-python
#from newsapi import NewsApiClient

def get_all_articles(source_id, api_key, max_articles=500, include_full_article_text=True):
    
    # init
    newsapi = NewsApiClient(api_key=api_key)
    
    print(source_id)
    
    all_article_objects = []
    page_number = 1
    
    while len(all_article_objects) < max_articles:
        page_number = page_number + 1
        response = newsapi.get_everything(sources=source_id,
                                      language='en',
                                      page=page_number, # can also use dates
                                        page_size=100) # 100 is maximum page size
    
        assert response['status'] == 'ok'
        
        all_article_objects += get_list_of_article_objects(response, include_full_articles=include_full_article_text)
        print("number of articles collected: %s" % len(all_article_objects[:max_articles]))
        
    return all_article_objects[:max_articles]
        
#
# Testing
#

# source
#selected_source_id = name_id_dict['Fox News']
    
# number of documents
#num_documents = 50

# example -- set include full article text to True for scrapping the actual site
#all_article_objects = get_all_articles(selected_source_id, api_key, max_articles=num_documents, include_full_article_text=False)


In [126]:
# turn list of article objects into single giant corpus

def article_objects_to_single_document_strings(article_objects):
    """combines title, description and text of all articles into single string"""
    
    concatinated_article_objects = []
    for ao in article_objects:
        article_string = ' '
        title = ao['title']
        description = ao['description']
        text = ''
        if 'text' in ao:
            text = ao['text']
            if text == None:
                text = ''
                
        article_string += title + '. ' + description + '. ' + text + '. '
        concatinated_article_object = {
            'id' : ao['news_source_id'],
            'article_string' : article_string
        }
        concatinated_article_objects.append(concatinated_article_object)
    
    return concatinated_article_objects
        
#
# Testing
#

# source
#selected_source_id = name_id_dict['Fox News']
    
# number of documents
#num_documents = 50

# example -- set include full article text to True for scrapping the actual site
#all_article_objects = get_all_articles(selected_source_id, api_key, max_articles=num_documents, include_full_article_text=True)

#simple_article_objects = article_objects_to_single_document_strings(all_article_objects)
#simple_article_objects

In [127]:
#
# Selected sources and bias labels (conservative, nuetral, liberal)
# 
# Selected sources 'Breitbart News', 'Fox News', 'Reuters', 'MSNBC', and 'CNN' - (conservative, conservative, neutral, liberal, liberal)
#
selected_source_ids = ['cnn', 'msnbc', 'reuters', 'fox-news', 'breitbart-news']

# bias labels python dictonary
id_bias_dict = {
    'cnn' : 'liberal',
    'msnbc' : 'liberal',
    'reuters' : 'neutral',
    'fox-news' : 'conservative',
    'breitbart-news' : 'conservative'
}

id_bias_dict

# pandas data frame version
id_bias_df = pd.DataFrame.from_dict(data=id_bias_dict, orient='index', columns=['bias'])
id_bias_df.index.name = 'id'
id_bias_df

# join name id dataframe and bias id dataframe
selected_news_sources_df = pd.merge(id_name_df, id_bias_df, on='id')
selected_news_sources_df

,name,bias
id,,
breitbart-news,Breitbart News,conservative
cnn,CNN,liberal
fox-news,Fox News,conservative
msnbc,MSNBC,liberal
reuters,Reuters,neutral


In [161]:
#
# scrape from selected sources
#
from sklearn.feature_extraction.text import CountVectorizer


# sources
selected_source_ids = list(selected_news_sources_df.index.values)
    
# number of documents
num_documents_per_source = 200

def scrape_from_sources_bag_of_words(api_key, selected_source_ids, num_documents_per_source):
    
 
    frames = []
   
    for sid in selected_source_ids:
        
        #set include full article text to True for scrapping the actual site
        all_article_objects = get_all_articles(sid, api_key, max_articles=num_documents_per_source, include_full_article_text=False)

        simple_article_objects = article_objects_to_single_document_strings(all_article_objects)
        source_articles = [x['article_string'] for x in simple_article_objects]

        #count_vect = CountVectorizer()
        #bag_words = count_vect.fit_transform(source_articles)
        count_vect = CountVectorizer(input=source_articles, stop_words='english')
        count_vect.fit_transform(source_articles)
        
        
        vocab_df = pd.DataFrame.from_dict(data=count_vect.vocabulary_, orient='index', columns=['frequency'])
        vocab_df.index.name = 'term'
        vocab_df['id'] = sid
        
        # reset
        #%reset_selective -f "^count_vect$"
        #del count_vect
        #print(count_vect.vocabulary_)
        
        frames.append(vocab_df)
    
    source_id_vocab_df = pd.concat(frames)
    
    return source_id_vocab_df
    

# scrape from all selected sources and store as term frequency id in panda dataframe
sources_bag_of_words_df = scrape_from_sources_bag_of_words(api_key, selected_source_ids, num_documents_per_source)

sources_bag_of_words_df

breitbart-news
number of articles collected: 100
number of articles collected: 200
cnn
number of articles collected: 98
number of articles collected: 198
number of articles collected: 200
fox-news
number of articles collected: 100
number of articles collected: 200
msnbc
number of articles collected: 100
number of articles collected: 200
reuters
number of articles collected: 100
number of articles collected: 200


,frequency,id
term,,
mexican,1403,breitbart-news
cartel,368,breitbart-news
boss,291,breitbart-news
offers,1535,breitbart-news
100k,4,breitbart-news
assassination,180,breitbart-news
breitbart,302,breitbart-news
texas,2206,breitbart-news
writer,2436,breitbart-news


In [162]:
# print out n most common words by source
n = 20
df_grouped = sources_bag_of_words_df.groupby(by='id')
for sid,grp in df_grouped:

    top_words = grp.sort_values(by='frequency')[-n:]
    print('Top %s words in %s source.' % (n, sid))
    print(top_words)

Top 20 words in breitbart-news source.
          frequency              id
term                               
worry          2429  breitbart-news
worse          2430  breitbart-news
worsened       2431  breitbart-news
worst          2432  breitbart-news
worth          2433  breitbart-news
wouldn         2434  breitbart-news
wowing         2435  breitbart-news
writer         2436  breitbart-news
writers        2437  breitbart-news
wrong          2438  breitbart-news
wrote          2439  breitbart-news
year           2440  breitbart-news
years          2441  breitbart-news
yelling        2442  breitbart-news
york           2443  breitbart-news
yorker         2444  breitbart-news
yorkers        2445  breitbart-news
yuma           2446  breitbart-news
zombie         2447  breitbart-news
zucker         2448  breitbart-news
Top 20 words in cnn source.
           frequency   id
term                     
worldwide       2162  cnn
worried         2163  cnn
worth           2164  cnn
wows       